In [14]:
import numpy as np
import time 


np.random.seed(69)

# Basic optimization 
Let's start our discussion with the simple example of optimizing matrix multiplication calculation

In [2]:
N, M, K = 100, 50, 75
A = np.random.rand(N, M)
B = np.random.rand(M, K)
C = np.zeros((N, K))

In [3]:
%%timeit -r 10 -n 1

for i in range(N):
     for j in range(K):
         for n in range(M):
             C[i][j]+=A[i][n]*B[n][j]

223 ms ± 17.6 ms per loop (mean ± std. dev. of 10 runs, 1 loop each)


In [4]:
%%timeit -r 10 -n 1

for n in range(M):
    for i in range(N):
        for j in range(K):
             C[i][j]+=A[i][n]*B[n][j]

224 ms ± 23.4 ms per loop (mean ± std. dev. of 10 runs, 1 loop each)


In [5]:
%%timeit -r 100 -n 1

C = A @ B

The slowest run took 4.53 times longer than the fastest. This could mean that an intermediate result is being cached.
111 µs ± 34.7 µs per loop (mean ± std. dev. of 100 runs, 1 loop each)


$\text{Example of a python deadlock:}$

```python
import threading

lock1 = threading.Lock()
lock2 = threading.Lock()

def thread1():
    with lock1:
        with lock2:  # Deadlock if thread2 holds lock2
            print("Thread1")

def thread2():
    with lock2:
        with lock1:  # Deadlock if thread1 holds lock1
            print("Thread2")

t1 = threading.Thread(target=thread1)
t2 = threading.Thread(target=thread2)
t1.start(); t2.start()
```

In [47]:
query =\
r"""
#include <stdio.h>
#include <stdlib.h>
#include <omp.h>

int main(int argc, char** argv) {

  int num_threads = 1;
  // Usage:
  if (argc < 2) {
      printf("Usage: exec param");
      return 1;
  }
  else {
      // test that argument is integer
      num_threads = atoi(argv[1]);
  }

  omp_set_num_threads(num_threads);
  #pragma omp parallel
  {
      printf("Hello from threadnum = %d\n", omp_get_thread_num());
  }

  return 0;
}

"""
open(f"prog.c", 'w').write(query)
!gcc -fopenmp .\prog.c -o exec & .\exec.exe 16

Hello from threadnum = 5
Hello from threadnum = 4
Hello from threadnum = 9
Hello from threadnum = 8
Hello from threadnum = 6
Hello from threadnum = 12
Hello from threadnum = 7
Hello from threadnum = 11
Hello from threadnum = 10
Hello from threadnum = 14
Hello from threadnum = 1
Hello from threadnum = 13
Hello from threadnum = 15
Hello from threadnum = 0
Hello from threadnum = 2
Hello from threadnum = 3


In [48]:
query =\
r"""
#include <stdio.h>
#include <stdlib.h>
#include <omp.h>

#define N 10

void pretty_print(int* array, int num_elems) {
    for (int n = 0; n < num_elems; ++n) {
        printf("%d\t", array[n]);
    }
    printf("\n");
}

int main(int argc, char** argv) {

  int num_threads = 1;
  // Usage:
  if (argc < 2) {
      printf("Usage: exec param");
      return 1;
  }
  else {
      // test that argument is integer
      num_threads = atoi(argv[1]);
  }

  omp_set_num_threads(num_threads);

  int a[N];
  int b[N];
  int c[N];
  int sum = 0;
  double end = 0.;
  double start = omp_get_wtime();

  #pragma omp parallel default(none) shared(a,b,c,sum)
  {
      #pragma omp for nowait
      for(int i=0; i<N; ++i) {
          a[i] = i;
          b[i] = i;
          c[i] = 0;
      }

      #pragma omp for
      for(int i=0; i<N; ++i) {
          c[i] = a[i] + b[i];
      }

      #pragma omp for reduction(+:sum)
      for(int i=0; i<N; ++i) {
          sum = sum + c[i];
      }
  }
  end = omp_get_wtime();


  pretty_print(a, N);
  pretty_print(b, N);
  pretty_print(c, N);
  printf("sum = %d\t elapsed time = %e ms\n", sum, (end-start)*1e3);

  return 0;
}

"""
open(f"prog.c", 'w').write(query)
!gcc -fopenmp .\prog.c -o exec_2 & .\exec_2.exe 4

0	1	2	3	4	5	6	7	8	9	
0	1	2	3	4	5	6	7	8	9	
0	2	4	6	8	10	12	14	16	18	
sum = 90	 elapsed time = 1.000166e+00 ms
